### 데이터 전처리

In [2]:
from datasets import load_dataset
dataset = load_dataset("wikipedia", "20220301.en", split="train")

# 캐시된 경로 확인
print(dataset.cache_files)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'filename': '/home/zeus/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/d41137e149b2ea90eead07e7e3f805119a8c22dd1d5b61651af8e3e3ee736001/wikipedia-train-00000-of-00041.arrow'}, {'filename': '/home/zeus/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/d41137e149b2ea90eead07e7e3f805119a8c22dd1d5b61651af8e3e3ee736001/wikipedia-train-00001-of-00041.arrow'}, {'filename': '/home/zeus/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/d41137e149b2ea90eead07e7e3f805119a8c22dd1d5b61651af8e3e3ee736001/wikipedia-train-00002-of-00041.arrow'}, {'filename': '/home/zeus/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/d41137e149b2ea90eead07e7e3f805119a8c22dd1d5b61651af8e3e3ee736001/wikipedia-train-00003-of-00041.arrow'}, {'filename': '/home/zeus/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/d41137e149b2ea90eead07e7e3f805119a8c22dd1d5b61651af8e3e3ee736001/wikipedia-train-00004-of-00041.arrow'}, {'filename': '/home/zeus/.cache/huggingface/datasets/wikipedia/202203

In [1]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_de = spacy.load('de_core_news_sm') # 독일어 토큰화(tokenization)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

In [2]:
# 간단히 토큰화(tokenization) 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


- 영어 및 독일어 토큰화 함수 정의

In [3]:
# 독일어(Deutsch) 문장을 토큰화 하는 함수 (순서를 뒤집지 않음)
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

- 데이터셋 로드
  - huggingface 데이터 다운로드 중, timeout 에러가 발생하여 강제로 60초로 증강

In [4]:
from datasets import load_dataset
import huggingface_hub

# 타임아웃을 60초로 늘리기
huggingface_hub.constants.HF_HUB_HTTP_TIMEOUT = 60

# 다시 시도
dataset = load_dataset("bentrevett/multi30k")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']

In [6]:
from torchtext.data.utils import get_tokenizer

In [7]:
from datasets import load_dataset
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import huggingface_hub

In [8]:
train_dataset

Dataset({
    features: ['en', 'de'],
    num_rows: 29000
})

In [9]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [10]:
# Tokenizer 준비 (spacy 설치 필요)
tokenizer_de = get_tokenizer("spacy", language="de_core_news_sm")
tokenizer_en = get_tokenizer("spacy", language="en_core_web_sm")

# 특수 토큰 정의
specials = ["<unk>", "<pad>", "<sos>", "<eos>"]

# Vocab 생성용 generator 함수 정의
def yield_tokens(data, lang, tokenizer):
    for example in data:
        yield tokenizer(example[lang].lower())

# SRC vocab (독일어)
vocab_src = build_vocab_from_iterator(
    yield_tokens(train_dataset, "de", tokenizer_de),
    specials=specials,
    min_freq=2
)
vocab_src.set_default_index(vocab_src["<unk>"])

# TRG vocab (영어)
vocab_trg = build_vocab_from_iterator(
    yield_tokens(train_dataset, "en", tokenizer_en),
    specials=specials,
    min_freq=2
)
vocab_trg.set_default_index(vocab_trg["<unk>"])

# 결과 출력
print(f"len(SRC): {len(vocab_src)}")
print(f"len(TRG): {len(vocab_trg)}")


len(SRC): 7853
len(TRG): 5893


In [11]:
vocab_trg.vocab

In [12]:
print(vocab_trg.vocab["abcabc"]) # 없는 단어: 0
print(vocab_trg.vocab["<pad>"]) # 패딩(padding): 1
print(vocab_trg.vocab["<sos>"]) # <sos>: 2
print(vocab_trg.vocab["<eos>"]) # <eos>: 3
print(vocab_trg.vocab["hello"])
print(vocab_trg.vocab["world"])

0
1
2
3
4111
1752


- 데이터 로더 생성

In [13]:
import torch
from torch.utils.data import DataLoader

BATCH_SIZE = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
import torch
from torch.nn.utils.rnn import pad_sequence

def process(example, src_tokenizer, trg_tokenizer, vocab_src, vocab_trg):
    src_tokens = ["<sos>"] + src_tokenizer(example['de'].lower()) + ["<eos>"]
    trg_tokens = ["<sos>"] + trg_tokenizer(example['en'].lower()) + ["<eos>"]

    src_indices = [vocab_src[token] for token in src_tokens]
    trg_indices = [vocab_trg[token] for token in trg_tokens]

    return torch.tensor(src_indices), torch.tensor(trg_indices)

def collate_fn(batch):
    src_batch, trg_batch = zip(*[
        process(example, tokenizer_de, tokenizer_en, vocab_src, vocab_trg)
        for example in batch
    ])

    src_batch = pad_sequence(src_batch, padding_value=vocab_src["<pad>"], batch_first=True)
    trg_batch = pad_sequence(trg_batch, padding_value=vocab_trg["<pad>"], batch_first=True)

    return src_batch.to(device), trg_batch.to(device)


In [15]:
def collate_fn(batch):
    src_batch, trg_batch = zip(*[
        process(example, tokenizer_de, tokenizer_en, vocab_src, vocab_trg)
        for example in batch
    ])

    src_batch = pad_sequence(src_batch, padding_value=vocab_src["<pad>"], batch_first=True)
    trg_batch = pad_sequence(trg_batch, padding_value=vocab_trg["<pad>"], batch_first=True)

    return src_batch.to(device), trg_batch.to(device)

In [16]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [17]:
for i, batch in enumerate(train_loader):
    src = batch[0]
    trg = batch[1]

    print(f"첫 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[1]):
        print(f"인덱스 {i}: {src[0][i].item()}") # 여기에서는 [Seq_num, Seq_len]

    # 첫 번째 배치만 확인
    break

첫 번째 배치 크기: torch.Size([128, 27])
인덱스 0: 2
인덱스 1: 18
인덱스 2: 26
인덱스 3: 253
인덱스 4: 30
인덱스 5: 84
인덱스 6: 20
인덱스 7: 88
인덱스 8: 7
인덱스 9: 15
인덱스 10: 110
인덱스 11: 7647
인덱스 12: 3171
인덱스 13: 4
인덱스 14: 3
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1


### Multi-head attention
- 어텐션 입력값
  - q: 쿼리
  - k: 키
  - v: 값
- 하이퍼 파라미터
  - hidden_dim: 하나의 단어에 대한 임베딩 차원
  - n_heads: 헤드의 갯수(scaled dot-product attention)
  - dropout_ratio: 드롭아웃의 비율

### 구현해야하는 것
- Multi-head attention
  - hidden_dim = head_dim * n_heads

In [18]:
from torch import nn

In [19]:
class MultiheadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
        super().__init__()

        self.device = device
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads
        self.head_dim = hidden_dim // n_heads
        self.dropout_ratio = dropout_ratio

        self.q_linear = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.k_linear = nn.Linear(hidden_dim, hidden_dim, bias=False)   
        self.v_linear = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.out_linear = nn.Linear(hidden_dim, hidden_dim, bias=False)

        self.dropout = nn.Dropout(dropout_ratio)


    def forward(self, q, k, v, mask=None):

        batch_size, seq_len, hidden_dim = q.shape

        # q: [batch_size, query_len, hidden_dim]
        # k: [batch_size, key_len, hidden_dim]
        # v: [batch_size, value_len, hidden_dim]
        # query_len, key_len, value_len은 모두 seq_len과 같음

        Q = self.q_linear(q) 
        K = self.k_linear(k)
        V = self.v_linear(v)

        # [batch_size, seq_len, hidden_dim] -> [batch_size, n_heads, seq_len, head_dim] 으로 조정 필요

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3) 

        # Attention(Q, K, V) = softmax(QK^T / sqrt(d_k))V

        d_k = torch.sqrt(torch.FloatTensor([self.head_dim])).to(self.device)

        # energy: [batch_size, n_heads, query_len, key_len]
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / d_k

        # 마스크(mask)를 사용하는 경우
        if mask is not None:
            # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기
            energy = energy.masked_fill(mask==0, -1e10)

        # attention: [batch_size, n_heads, seq_len, head_dim]
        attention_scores = torch.softmax(energy, dim=-1)
        attention = torch.matmul(attention_scores, V)

        # attention: [batch_size, n_heads, seq_len, head_dim] -> x: [batch_size, seq_len, n_heads, head_dim]
        x = attention.permute(0, 2, 1, 3).contiguous()

        # x: [batch_size, seq_len, hidden_dim]
        x = x.view(batch_size, -1, self.hidden_dim)

        return x, attention_scores

In [20]:
class FeedForwardLayer(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_ratio=0.1):
        super().__init__()

        self.linear_1 = nn.Linear(hidden_dim, pf_dim, bias=False)
        self.linear_2 = nn.Linear(pf_dim, hidden_dim, bias=False)
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):
        x = self.linear_1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.linear_2(x)

        return x

### Encoder 아키텍쳐

In [21]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        # input and ouput have same dimension
        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiheadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.feedforward = FeedForwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, src, src_mask):
        _src, _ = self.self_attention(src, src, src, src_mask)
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        _src = self.feedforward(src)
        src = self.ff_layer_norm(src + self.dropout(_src))

        return src

In [22]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()
        
        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src, src_mask):
        # src: [batch_size, src_len]
        # src_mask: [batch_size, src_len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        # pos: [batch_size, src_lne]
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        for layer in self.layers:
            src = layer(src, src_mask)

        return src

### 디코더 아키텍쳐

In [23]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attention_1 = MultiheadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.self_attn_layer_norm_1 = nn.LayerNorm(hidden_dim)
        self.self_attention_2 = MultiheadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.self_attn_layer_norm_2 = nn.LayerNorm(hidden_dim)

        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.feedforward = FeedForwardLayer(hidden_dim, pf_dim, dropout_ratio)
        
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        _trg, _ = self.self_attention_1(trg, trg, trg, trg_mask)
        trg = self.self_attn_layer_norm_1(trg + self.dropout(_trg))

        _trg, attention_score = self.self_attention_2(trg, enc_src, enc_src, src_mask)
        trg = self.self_attn_layer_norm_2(trg + self.dropout(_trg))

        _trg = self.feedforward(trg)
        trg = self.feedforward(trg + self.dropout(_trg))

        return trg, attention_score

In [24]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])  

        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(device)

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        # trg: [batch_size, trg_len, hideen_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        output = self.fc_out(trg)

        # output: [batch_size, trg_len, output_dim]

        return output, attention

### 트랜스포머 구현

In [25]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        # src: [batch_size, src_len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask = [batch_size, 1, 1, src_len]

        return src_mask

    def make_trg_mask(self, trg):

        # trg: [batch_size, trg_len]

        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

        trg_len = trg.shape[1]

        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=self.device)).bool()

        trg_mask = trg_pad_mask & trg_sub_mask

        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask

    def forward(self, src, trg):

        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        enc_src = self.encoder(src, src_mask)

        # enc_src: [batch_size, src_len, hidden_dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention

### 학습 코드

In [26]:
INPUT_DIM = len(vocab_src)
OUTPUT_DIM = len(vocab_trg)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [27]:
vocab_src.vocab["<pad>"]

SRC_PAD_IDX = vocab_src.vocab["<pad>"]
TRG_PAD_IDX = vocab_trg.vocab["<pad>"]

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,024,517 trainable parameters


In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(7853, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0-2): 3 x EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiheadAttentionLayer(
          (q_linear): Linear(in_features=256, out_features=256, bias=False)
          (k_linear): Linear(in_features=256, out_features=256, bias=False)
          (v_linear): Linear(in_features=256, out_features=256, bias=False)
          (out_linear): Linear(in_features=256, out_features=256, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feedforward): FeedForwardLayer(
          (linear_1): Linear(in_features=256, out_features=512, bias=False)
          (linear_2): Linear(in_features=512, out_features=256, bias=False)
          (dropout): Dropout(p=0.1, inplace=False

In [30]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [31]:
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch[0]
        trg = batch[1]

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스()는 제외
        # 입력을 할 때는 부터 시작하도록 처리
        output, _ = model(src, trg[:,:-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0()은 제외
        trg = trg[:,1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch[0]
            trg = batch[1]

            # 출력 단어의 마지막 인덱스()는 제외
            # 입력을 할 때는 부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0()은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


- 학습 진행

In [33]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 0m 10s
	Train Loss: 4.312 | Train PPL: 74.599
	Validation Loss: 3.170 | Validation PPL: 23.797
Epoch: 02 | Time: 0m 9s
	Train Loss: 2.881 | Train PPL: 17.837
	Validation Loss: 2.440 | Validation PPL: 11.478
Epoch: 03 | Time: 0m 9s
	Train Loss: 2.331 | Train PPL: 10.290
	Validation Loss: 2.150 | Validation PPL: 8.588
Epoch: 04 | Time: 0m 9s
	Train Loss: 2.019 | Train PPL: 7.529
	Validation Loss: 2.005 | Validation PPL: 7.428
Epoch: 05 | Time: 0m 9s
	Train Loss: 1.813 | Train PPL: 6.127
	Validation Loss: 1.922 | Validation PPL: 6.834
Epoch: 06 | Time: 0m 9s
	Train Loss: 1.660 | Train PPL: 5.258
	Validation Loss: 1.894 | Validation PPL: 6.644
Epoch: 07 | Time: 0m 9s
	Train Loss: 1.539 | Train PPL: 4.660
	Validation Loss: 1.901 | Validation PPL: 6.692
Epoch: 08 | Time: 0m 9s
	Train Loss: 1.443 | Train PPL: 4.234
	Validation Loss: 1.881 | Validation PPL: 6.562
Epoch: 09 | Time: 0m 10s
	Train Loss: 1.363 | Train PPL: 3.907
	Validation Loss: 1.881 | Validation PPL: 6.560
Epo

### 테스트

In [ ]:
model.load_state_dict(torch.load('transformer_german_to_english.pt'))

test_loss = evaluate(model, test_loader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 1.919 | Test PPL: 6.812


In [36]:
vocab_src.get_itos()

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 '.',
 'ein',
 'einem',
 'in',
 'eine',
 ',',
 'und',
 'mit',
 'auf',
 'mann',
 'einer',
 'der',
 'frau',
 'die',
 'zwei',
 'einen',
 'im',
 'an',
 'von',
 'sich',
 'dem',
 'mädchen',
 'junge',
 'vor',
 'zu',
 'steht',
 'männer',
 'sitzt',
 'hund',
 'den',
 'straße',
 'während',
 'gruppe',
 'hält',
 'spielt',
 'das',
 'hemd',
 'personen',
 'über',
 'drei',
 'eines',
 'frauen',
 'blauen',
 'neben',
 'ist',
 'kind',
 'roten',
 'weißen',
 'stehen',
 'sitzen',
 'menschen',
 'am',
 'aus',
 'spielen',
 'durch',
 'bei',
 'geht',
 'trägt',
 'fährt',
 'wasser',
 'um',
 'kinder',
 'kleines',
 'person',
 'macht',
 'springt',
 'kleiner',
 'schwarzen',
 'entlang',
 'leute',
 'gehen',
 'etwas',
 'mehrere',
 'seinem',
 'großen',
 'oberteil',
 'jungen',
 'hand',
 'grünen',
 'läuft',
 'sind',
 'für',
 'hintergrund',
 'fahrrad',
 'freien',
 'jacke',
 'luft',
 'strand',
 'ball',
 'hat',
 'anderen',
 'schaut',
 'junger',
 'kleidung',
 'hinter',
 'sie',
 'nach',
 'an

In [37]:
# 번역(translation) 함수
def translate_sentence(sentence, vocab_src, vocab_trg, model, device, max_len=50, logging=True):
    model.eval() # 평가 모드

    init_token = "<sos>"
    eos_token = "<eos>"

    if isinstance(sentence, str):
        nlp = spacy.load('de_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에  토큰, 마지막에  토큰 붙이기
    tokens = [init_token] + tokens + [eos_token]
    if logging:
        print(f"전체 소스 토큰: {tokens}")

    src_indexes = [vocab_src[token] for token in tokens]
    if logging:
        print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # 소스 문장에 따른 마스크 생성
    src_mask = model.make_src_mask(src_tensor)

    # 인코더(endocer)에 소스 문장을 넣어 출력 값 구하기
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    # 처음에는  토큰 하나만 가지고 있도록 하기
    trg_indexes = [vocab_trg[init_token]]

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        # 출력 문장에 따른 마스크 생성
        trg_mask = model.make_trg_mask(trg_tensor)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)

        # 출력 문장에서 가장 마지막 단어만 사용
        pred_token = output.argmax(2)[:,-1].item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기
        # 를 만나는 순간 끝
        if pred_token == vocab_trg[eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [vocab_trg.get_itos()[i] for i in trg_indexes]

    # 첫 번째 는 제외하고 출력 문장 반환
    return trg_tokens[1:], attention

In [38]:
test_dataset["en"][0]

'A man in an orange hat starring at something.'

In [39]:
vocab_src.get_itos()[1]

'<pad>'

In [40]:
# example_idx = 23

# src = test_dataset["de"][example_idx]
# trg = test_dataset["en"][example_idx]

src = "Erzählen Sie mir die große Neuigkeit des Tages." # Tell me the big news of the day.

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')

translation, attention = translate_sentence(src, vocab_src, vocab_trg, model, device, logging=True)

print("모델 출력 결과:", " ".join(translation))

소스 문장: Erzählen Sie mir die große Neuigkeit des Tages.
타겟 문장: tensor([[  2,  16,  24,  ...,   1,   1,   1],
        [  2, 113,  30,  ...,   1,   1,   1],
        [  2,   4,  53,  ...,   1,   1,   1],
        ...,
        [  2,  16,  50,  ...,   1,   1,   1],
        [  2,   4,  38,  ...,   1,   1,   1],
        [  2,   4,  14,  ...,   1,   1,   1]], device='cuda:0')
전체 소스 토큰: ['<sos>', 'erzählen', 'sie', 'mir', 'die', 'große', 'neuigkeit', 'des', 'tages', '.', '<eos>']
소스 문장 인덱스: [2, 0, 99, 6783, 17, 168, 0, 151, 4140, 4, 3]
모델 출력 결과: <unk> looking at the large <unk> of the daytime . <eos>
